In [1]:
# This file aims to debug the newly created solvent diffusion model with EC drag (add one submodel variable for EC concentration)

In [2]:
import pybamm as pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.700000000000002e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 )  

In [4]:
Model_0  = pybamm.lithium_ion.DFN(options={"solvent diffusion": "EC" }) 
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);

Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] =   "[input]";
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   "[input]";
Para_0['EC transference number'] =   "[input]";
Para_0['EC diffusivity [m2.s-1]'] =   "[input]";
Para_0['Electrolyte diffusivity [m2.s-1]'] =   "[input]";

Para_0.search("diffusivity")
gamma_e_ec_Rio = Para_0.evaluate(Model_0.param.gamma_e_ec_Rio)  
print(gamma_e_ec_Rio)

In [5]:
Sim_0    = pybamm.Simulation(
    Model_0, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve(
    inputs={
        "EC transference number":-1.4,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "Typical EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "EC diffusivity in electrolyte [m2.s-1]":5E-12,
        "Electrolyte diffusivity [m2.s-1]":3E-10,
        });


In [6]:
Sol_1    = Sim_0.solve(
    inputs={
        "EC transference number":-1.1,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "Typical EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "EC diffusivity in electrolyte [m2.s-1]":5E-14,
        "Electrolyte diffusivity [m2.s-1]":3E-10,
    });

In [7]:
Sol_2    = Sim_0.solve(
    inputs={
        "EC transference number":-1.4,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "Typical EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "EC diffusivity in electrolyte [m2.s-1]":5E-12,
        "Electrolyte diffusivity [m2.s-1]":9E-10,
    });


In [8]:
label = ["Dec=5E-12,De=3E-10","Dec=5E-14,De=3E-10","Dec=5E-12,De=9E-10",] 
output_variables3 = [
    #"Terminal voltage [V]",   
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "Total EC in electrolyte [mol]",
    "Total lithium in electrolyte [mol]",
]
quick_plot = pybamm.QuickPlot([Sol_0,Sol_1,Sol_2], output_variables3,label,) #     variable_limits='tight'
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=4.152807959921794, step=0.041528079599217935…